In [95]:
# Import tools
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import sqlite3 as db
from datetime import date

### Scraping and Data Cleaning

In [9]:
# Load JSON from website and convert it for Beautiful Soup

In [91]:
names = []
positions = []
teams = []
games_played = []
mpg = []
orpm = []
drpm = []
rpm = []
wins = []

In [92]:
def scrape_table(url, names, positions, mpg, teams, orpm, drpm, rpm, wins):
    # Load html from website and convert it for Beautiful Soup
    rpm_statistics = requests.get(url)
    soup = BeautifulSoup(rpm_statistics.content, 'html.parser')
    # Isolate table
    table = soup.find('table', class_="tablehead")

    # Grab Rows
    rows = table.find_all('tr')

    # Extract Relevant Information
    for row in rows:
        stats = row.find_all('td')
        # Parse Out Position
        name_and_position = stats[1].text
        if name_and_position[len(name_and_position)-1] == 'C':
            positions.append('C')
            names.append(name_and_position[0:len(name_and_position)-3])
        else:
            positions.append(name_and_position[len(name_and_position)-2: len(name_and_position)])
            names.append(name_and_position[0:len(name_and_position)-4])
        teams.append(stats[2].text)
        games_played.append(stats[3].text)
        mpg.append(stats[4].text)
        orpm.append(stats[5].text)
        drpm.append(stats[6].text)
        rpm.append(stats[7].text)
        wins.append(stats[8].text)

In [93]:
# Repeat Scraping Process For All Players
pages = 11

scrape_table('http://www.espn.com/nba/statistics/rpm', names, positions, mpg, teams, orpm, drpm, rpm, wins)

for i in range(1, 11):
    scrape_table(f'http://www.espn.com/nba/statistics/rpm/_/page/{i}', names, positions, mpg, teams, orpm, drpm, rpm, wins)

In [94]:
# Create a Pandas Dataframe with lists

scraped_stats = pd.DataFrame({'Player Name': names, 'Position': positions, 'Team': teams, 'MPG': mpg, 'ORPM': orpm, 'DRPM':drpm, 'RPM': rpm, 'Wins': wins})

# Drop First Index (Technically isn't the first player)
scraped_stats = scraped_stats.iloc[1: , :]
scraped_stats


,Player Name,Position,Team,MPG,ORPM,DRPM,RPM,Wins
1,Luka Doncic,PG,DAL,36.9,7.99,1.90,9.89,4.85
2,Stephen Curry,PG,GS,34.8,8.87,0.63,9.50,4.84
3,Jayson Tatum,SF,BOS,37.2,6.00,2.37,8.37,4.42
4,Nikola Jokic,C,DEN,31.1,4.66,1.96,6.62,2.57
5,Giannis Antetokounmpo,PF,MIL,32.8,5.17,1.28,6.45,2.90
...,...,...,...,...,...,...,...,...
446,Max Christi,G,LAL,11.5,-3.72,-0.17,-3.89,-0.06
447,Nerlens Noel,C,DET,12.0,-4.27,0.36,-3.90,-0.04
448,Terry Taylor,SF,IND,10.4,-2.60,-1.31,-3.91,-0.08
449,A.J. Gree,G,MIL,6.0,-2.99,-0.94,-3.92,-0.02


In [110]:
# Store DF in Sqlite DB
today = date.today()
today = today.strftime("%b_%d_%Y")

conn = db.connect('rpm.sqlite')
scraped_stats.to_sql(f'RPM_Statistics_{today}', conn, if_exists='replace', index=False)

c:\Users\neilp\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\generic.py:2778: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [111]:
# Test table
pd.read_sql(f'select * from RPM_Statistics_{today}', conn)

,Player Name,Position,Team,MPG,ORPM,DRPM,RPM,Wins
0,Luka Doncic,PG,DAL,36.9,7.99,1.90,9.89,4.85
1,Stephen Curry,PG,GS,34.8,8.87,0.63,9.50,4.84
2,Jayson Tatum,SF,BOS,37.2,6.00,2.37,8.37,4.42
3,Nikola Jokic,C,DEN,31.1,4.66,1.96,6.62,2.57
4,Giannis Antetokounmpo,PF,MIL,32.8,5.17,1.28,6.45,2.90
...,...,...,...,...,...,...,...,...
445,Max Christi,G,LAL,11.5,-3.72,-0.17,-3.89,-0.06
446,Nerlens Noel,C,DET,12.0,-4.27,0.36,-3.90,-0.04
447,Terry Taylor,SF,IND,10.4,-2.60,-1.31,-3.91,-0.08
448,A.J. Gree,G,MIL,6.0,-2.99,-0.94,-3.92,-0.02
